In [1]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

In [81]:
from pymystem3 import Mystem

mystem = Mystem()

In [3]:
pd.set_option('display.max_colwidth', 1000)

In [4]:
PATH_TO_DATA = './data_1'

In [5]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

In [6]:
data = pd.concat([pd.read_json(file, lines=True, encoding='UTF-8') for file in files], axis=0, ignore_index=True)

In [7]:
data.shape

(1987, 5)

In [8]:
data.head(3)

,content,keywords,summary,title,url
0,"В среду состоялось отложенное заседание Совета по федеральным государственным образовательным стандартам (ФГОС) при Министерстве образования и науки РФ. Собрание должно было состояться еще в понедельник, но было перенесено по просьбе членов совета. И вот пришло сообщение, что общественники выразили согласие с позицией министерства. Новые ФГОСы приняты.\nНа вчерашнем заседании был принят ФГОС по начальной общеобразовательной школе. До 28 марта продлятся косультации по ФГОСам для средней школы.\nНапомним, что накануне Гильдия словесников разместила открытое письмо на имя министра образования и науки РФ Ольги Васильевой. По мнению авторов письма, новые ФГОСы грубо нарушают права детей, уже проучившихся по существующему стандарту до 6-го класса. Приняв новый стандарт, Министерство образования дает право контролирующим органам ловить детей на незнании большого списка произведений (235 за пять лет обучения). «Это исключает возможность полноценного их освоения, создает риск формального, п...","[школа, образовательные стандарты, литература, история, фгос]","Глава Минобрнауки считает, что в нездоровом ажиотаже вокруг новых образовательных стандартов виноваты издательства учебной литературы","Ольга Васильева обещала ""НГ"" не перегружать школьников",https://amp.ng.ru/?p=http://www.ng.ru/education/2018-03-22/8_7195_school.html
1,"Хорошо, когда красота в глазах смотрящего живет свободно или хотя бы занимает широкий угол зрения. Плохо было б, если б она вовсе не озаряла своим светом космическую темень пустоты зрачка. Слава богу, такое вряд ли возможно. \nА случается, что красота уходит. Почему вдруг? И куда она девается, когда в один из философских обходов своего организма вы, еще недавно гордый ее обладатель, обескураженно ее недосчитываетесь? \nВообразите: прелестнейшее из созданий – ваша кошка пластичнейшими движениями рвет банкноту за банкнотой, забирается на карниз по шелковой занавеске или отгрызает полпаспорта. Где, скажите, теперь красота этой кошки? Или другой пример – с зазнобой сердца. Предмет романтичнейших грез наконец-то садится с вами на заветную скамейку в парке – закат, пение птах… И тут он силой своего обаяния с оглушительным плюхом обрушивает вокруг вас красоту и гармонию столетних дубов, тополей и прочего. Где, спрашивается, красота момента? \nЕсли от сказки после того, как ее рассказали,...","[красота, законы]",О живительной пользе укорота при выборе между плохим и хорошим,У красоты собственные закон и воля,https://amp.ng.ru/?p=http://www.ng.ru/style/2018-03-19/8_7192_beauty.html
2,"Когда-то Леонид Юзефович написал книгу о монгольской эпопее барона Унгерна «Самодержец пустыни» – она стала интеллектуальным бестселлером и классикой жанра – документальный роман. В то время автор попутно изучал и историю вооруженного восстания в Якутии в 1922–1923 годах под руководством Анатолия Пепеляева. И вот теперь из «якутского» материала сложилась отдельная книга. Тема ее для нынешнего читателя поистине раритетна. Ведь воевавший где-то на самом краю страны Пепеляев практически забыт, притом что о борьбе с ним когда-то в СССР выходили статьи и книги. В памяти потомков, образно говоря, от Пепеляева остался только пепел.\nЮзефович воскрешает в памяти не только его военные дела, но и человеческие черты. Этот провинциальный интеллигент, неврастеник и фаталист, начал восстание, практически не имея шансов на успех. Однако силою недюжинной харизмы Пепеляев сумел собрать вокруг себя многих боевых офицеров, таежных охотников и недовольных новыми порядками аборигенов. Для своих 32 лет ...","[юзефович, гражданская война, пепеляев, якутия]",Крепость из тел и призрак независимой Якутии,Апокалиптический бунт,https://amp.ng.ru/?p=http://www.ng.ru/zavisimaya/2017-12-19/15_7139_bunt.html


In [9]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))

In [10]:
evaluate(data['keywords'], data['keywords'])

Precision -  1.0
Recall -  1.0
F1 -  1.0
Jaccard -  1.0


## Токенизация, удаление стоп-слов и нормализация.

In [11]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–1234567890'
stops = set(stopwords.words('russian'))

In [86]:
stops_y = ['алтухов', 'большой', 'бы', 'быть', 'в', 'весь', 'вот', 'все', 'всей', 'вы','г', 'говорить', 'год', 'да', 'для', 'до', 'еще', 'же', 'знать', 'и', 'из', 'к', 'как', 'который', 'мочь', 'мы', 'на', 'наш', 'не', 'него', 'нее', 'нет', 'них', 'но', 'о', 'один', 'она', 'они', 'оно', 'оный', 'от', 'ото', 'по', 'с', 'свой', 'себя', 'сказать', 'та', 'такой', 'только', 'тот', 'ты', 'у', 'что', 'это', 'этот', 'я']

In [13]:
stops_antela = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', 'один', 'два', 'три', 'четыре', 'пять', 'шесть', 'семь', 'восемь', 'девять', 'ноль']

In [14]:
stops_names_w = ['Августа', 'Августина', 'Авдотья', 'Аврелия', 'Аврея', 'Аврора', 'Агапа', 'Агапия', 'Агарь', 'Агита', 'Агафа', 'Агафоклия', 'Агафоника', 'Агафья', 'Агафия', 'Аглаида', 'Аглая', 'Агна', 'Агнесса', 'Агния', 'Аграфена', 'Агриппина', 'Ада', 'Аделаида', 'Аделина', 'Аделла', 'Адель', 'Адельфина', 'Адина', 'Адолия', 'Адриана', 'Аза', 'Азалия', 'Азелла', 'Аида', 'Акилина', 'Аксинья', 'Аксиния', 'Акулина', 'Алевтина', 'Александра', 'Александрина', 'Алексина', 'Алёна', 'Алина', 'Алиса', 'Алла', 'Алфея', 'Альберта', 'Альбертина', 'Альбина', 'Альвина', 'Альфия', 'Амалия', 'Амата', 'Амелфа', 'Анастасия', 'Анатолия', 'Ангела', 'Ангелика', 'Ангелина', 'Анджела', 'Андрея', 'Андрона', 'Андроника', 'Анжелика', 'Анисья', 'Анисия', 'Анна', 'Антигона', 'Антониана', 'Антонида', 'Антонина', 'Антония', 'Анфима', 'Анфиса', 'Анфия', 'Анфуса', 'Аполлинария', 'Аполлония', 'Апраксин', 'Апрелия', 'Апфия', 'Аргентея', 'Ариадна', 'Арина', 'Ария', 'Арминия', 'Арсения', 'Артемида', 'Артемия', 'Аста', 'Астра', 'Афанасия', 'Аэлита', 'Беата', 'Беатриса', 'Белла', 'Бенедикта', 'Берта', 'Бландина', 'Богдана', 'Божена', 'Болеслава', 'Борислава', 'Бояна', 'Бронислава', 'Валентина', 'Валенсия', 'Валерия', 'Ванда', 'Васёна', 'Василида', 'Василина', 'Василиса', 'Василия', 'Василла', 'Васса', 'Вацлава', 'Вевея', 'Велимира', 'Велислава', 'Венедикта', 'Венуста', 'Венцеслава', 'Вера', 'Вереника', 'Вероника', 'Вербния', 'Веселина', 'Веста', 'Вестита', 'Вива', 'Вивея', 'Вивиана', 'Видина', 'Викентия', 'Викторина', 'Виктбрия', 'Вила', 'Вилена', 'Виленина', 'Вильгельмина', 'Виолетта', 'Виргиния', 'Виринея', 'Вита', 'Виталика', 'Виталина', 'Виталия', 'Витольда', 'Влада', 'Владилена', 'Владимира', 'Владислава', 'Владлена', 'Воислава', 'Воля', 'Всеслава', 'Гала', 'Галата', 'Галатея', 'Гали', 'Галина', 'Галла', 'Галя', 'Гая', 'Геласия', 'Гемелла', 'Гемина', 'Гения', 'Геннадия', 'Геновефа', 'Генриетта', 'Георгина', 'Гера', 'Германа', 'Гертруда', 'Гея', 'Глафира', 'Гликерия', 'Глорибза', 'Голиндуха', 'Гонеста', 'Гонората', 'Горгония', 'Горислава', 'Гортензия', 'Градислава', 'Грета', 'Далила', 'Даная', 'Дарья', 'Дария', 'Дебора', 'Деена', 'Декабрена', 'Денесия', 'Денница', 'Дея', 'Диана', 'Дигна', 'Дина', 'Диодора', 'Дионина', 'Дия', 'Доброгнева', 'Добромила', 'Добромира', 'Доброслава', 'Доминика', 'Домитилла', 'Домна', 'Домника', 'Домникия', 'Домнина', 'Донара', 'Доната', 'Дора', 'Дорофея', 'Доса', 'Досифея', 'Дросида', 'Дуклида', 'Ева', 'Евангелина', 'Еванфия', 'Евгения', 'Евдокия', 'Евдоксия', 'Евлалия', 'Евлампия', 'Евмения', 'Евминия', 'Евника', 'Евникия', 'Евномия', 'Евпраксия', 'Евсевия', 'Евстафия', 'Евстолия', 'Евтихия', 'Евтропия', 'Евфалия', 'Евфимия', 'Евфросиния', 'Екатерина', 'Елена', 'Елизавета', 'Еликонида', 'Епистима', 'Епистимия', 'Ермиония', 'Ефимия', 'Ефимья', 'Ефросиния', 'Ефросинья', 'Жанна', 'Жозефина', 'Зара', 'Зарема', 'Зарина', 'Зари', 'Зарина', 'Звезда', 'Земфира', 'Зенона', 'Зина', 'Зинаида', 'Зиновия', 'Злата', 'Зоя', 'Ива', 'Иванна', 'Ида', 'Идея', 'Изабелла', 'Изида', 'Изольда', 'Илария', 'Илия', 'Ильина', 'Инга', 'Инесса', 'Инна', 'Иоанна', 'Иовилла', 'Иола', 'Иоланта', 'Ипполита', 'Ираида', 'Ирина', 'Ирма', 'Исидора', 'Ифигения', 'Ия', 'Каздоя', 'Казимира', 'Калерия', 'Калида', 'Калиса', 'Каллиникия', 'Каллиста', 'Каллисфения', 'Кама', 'Камилла', 'Кандида', 'Капитолина', 'Карина', 'Каролина', 'Касиния', 'Келестина', 'Керкира', 'Кетевань', 'Кикилия', 'Кима', 'Кира', 'Кириакия', 'Кириана', 'Кирьяна', 'Кирилла', 'Клавдия', 'Клара', 'Клариса', 'Клементина', 'Клеопатра', 'Конкордия', 'Констанция', 'Корнелия', 'Кристина', 'Ксанфиппа', 'Ксения', 'Купава', 'Лавиния', 'Лавра', 'Лада', 'Лариса', 'Лаура', 'Леда', 'Лейла', 'Лемира', 'Ленина', 'Леокадия', 'Леонида', 'Леонила', 'Леонина', 'Лебния', 'Лея', 'Лиана', 'Ливия', 'Лидия', 'Лилиана', 'Лилия', 'Лина', 'Лира', 'Лия', 'Лилия', 'Лонгина', 'Лора', 'Лота', 'Луиза', 'Лукерья', 'Лукиана', 'Лукия', 'Лукреция', 'Любава', 'Любовь', 'Любомила', 'Любомира', 'Людмила', 'Люцина', 'Люция', 'Мавра', 'Магда', 'Магдалина', 'Магна', 'Маина', 'Майя', 'Макрина', 'Максима', 'Малания', 'Малинья', 'Малина', 'Мальвина', 'Мамелфа', 'Манефа', 'Маргарита', 'Мариам', 'Мариамна', 'Мариана', 'Марианна', 'Марьина', 'Мариетта', 'Марина', 'Марионилла', 'Мария', 'Марья', 'Марка', 'Маркеллина', 'Маркиана', 'Марксина', 'Марлена', 'Марта', 'Мартина', 'Мартиниана', 'Марфа', 'Марья', 'Мария', 'Марьяна', 'Марианна', 'Мастридия', 'Матильда', 'Матрёна', 'Матрона', 'Мая', 'Медея', 'Мелания', 'Меланья', 'Мелитика', 'Меркурия', 'Мерона', 'Милана', 'Милена', 'Милица', 'Милия', 'Милослава', 'Милютина', 'Мина', 'Минна', 'Минодора', 'Мира', 'Миропия', 'Мирослава', 'Мирра', 'Митродора', 'Михайлина', 'Млада', 'Модеста', 'Моика', 'Моника', 'Мстислава', 'Муза', 'Нада', 'Надежда', 'Нана', 'Наркисса', 'Настасия', 'Настасья', 'Наталия', 'Наталья', 'Нелли', 'Ненила', 'Неонила', 'Нида', 'Ника', 'Нила', 'Нимфа', 'Нимфодора', 'Нина', 'Нинель', 'Новелла', 'Нонна', 'Ноэми', 'Ноябрина', 'Нунехия', 'Оксана', 'Октавия', 'Октябрина', 'Олдама', 'Оливия', 'Олимпиада', 'Олимпиодора', 'Олимпия', 'Ольга', 'Ольда', 'Офелия', 'Павла', 'Павлина', 'Паисия', 'Паллада', 'Паллидия', 'Пальмира', 'Параскева', 'Патрикия', 'Пелагея', 'Перегрина', 'Перпетуя', 'Петра', 'Петрина', 'Петронилла', 'Петрония', 'Пиама', 'Пинна', 'Плакида', 'Плакилла', 'Платонида', 'Победа', 'Полактия', 'Поликсена', 'Поликсения', 'Полина', 'Поплия', 'Правдина', 'Прасковья', 'Препедигна', 'Прискилла', 'Просдока', 'Пульхерия', 'Пульхерья', 'Рада', 'Радана', 'Радислава', 'Радмила', 'Радомира', 'Радосвета', 'Радослава', 'Радость', 'Раиса', 'Рафаила', 'Рахиль', 'Ревекка', 'Ревмира', 'Регина', 'Рема', 'Рената', 'Римма', 'Рипсимия', 'Роберта', 'Рогнеда', 'Роза', 'Розалина', 'Розалинда', 'Розалия', 'Розина', 'Роксана', 'Романа', 'Ростислава', 'Русина', 'Руслана', 'Руфина', 'Руфиниана', 'Руфь', 'Сабина', 'Савватия', 'Савелла', 'Савина', 'Саломея', 'Сильвия', 'Самона', 'Сарра', 'Сатира', 'Светислава', 'Светлана', 'Светозара', 'Святослава', 'Севастьяна', 'Северина', 'Секлетея', 'Секлетинья', 'Селена', 'Селестина', 'Селина', 'Серафима', 'Сибилла', 'Сильва', 'Сильвана', 'Сильвестра', 'Сильвия', 'Симона', 'Синклитикия', 'Сира', 'Слава', 'Снандулия', 'Снежана', 'Сола', 'Соломонида', 'Сосипатра', 'Софрония', 'Софья', 'София', 'Станислава', 'Стелла', 'Степанида', 'Стефанида', 'Стефания', 'Сусанна', 'Сюзанна', 'Тавифа', 'Таисия', 'Таисья', 'Тамара', 'Тарасия', 'Татьяна', 'Текуса', 'Тереза', 'Тигрия', 'Тихомира', 'Тихослава', 'Тома', 'Томила', 'Транквиллина', 'Трифена', 'Трофима', 'Улита', 'Ульяна', 'Урбана', 'Урсула', 'Устина', 'Устиния', 'Устинья', 'Фабиана', 'Фавста', 'Фавстина', 'Фаина', 'Фантика', 'Феврония', 'Февронья', 'Федоза', 'Федора', 'Федосия', 'Федосья', 'Федотия', 'Федотья', 'Федула', 'Фёкла', 'Фекуса', 'Феликса', 'Фелица', 'Фелицата', 'Фелициана', 'Фелицитата', 'Фелиция', 'Феогния', 'Феодора', 'Феодосия', 'Феодота', 'Феодотия', 'Феодула', 'Феодулия', 'Феозва', 'Феоктиста', 'Феона', 'Феонилла', 'Фебния', 'Феопистия', 'Феосовия', 'Феофания', 'Феофила', 'Фервуфа', 'Фессалоника', 'Фессалоникия', 'Фетиния', 'Фетинья', 'Фея', 'Фива', 'Фивея', 'Филарета', 'Филиппа', 'Филиппин', 'Филомена', 'Филонилла', 'Филофея', 'Фиста', 'Флавия', 'Флёна', 'Флора', 'Флорентина', 'Флоренция', 'Флориана', 'Флорида', 'Фомаида', 'Фортуната', 'Фотина', 'Фотиния', 'Фотинья', 'Франциска', 'Фрида', 'Фридерика', 'Хаврония', 'Хариесса', 'Хариса', 'Харита', 'Харитина', 'Хиония', 'Хриса', 'Хрисия', 'Христиана', 'Христина', 'Цвета', 'Цветана', 'Целестина', 'Цецилия', 'Шарлотта', 'Шушаника', 'Эвелина', 'Эгина', 'Эдит', 'Элеонора', 'Элисса', 'Элла', 'Эллада', 'Эллина', 'Элоиза', 'Эльвира', 'Эмилиана', 'Эмилия', 'Эмма', 'Эннафа', 'Эра', 'Эрнеста', 'Эрнестина', 'Эсмеральда', 'Эсфирь', 'Юдифь', 'Юлиана', 'Юлиания', 'Юлия', 'Юния', 'Юнона', 'Юрия', 'Юстина', 'Ядвига', 'Яна', 'Янина', 'Ярослава'] 
stops_names_m = ['Абакум', 'Абрам', 'Абросим', 'Аввакум', 'Август', 'Авдей', 'Авдий', 'Авель', 'Авенир', 'Аверий', 'Аверкий', 'Аверьян', 'Авксентий', 'Аврам', 'Аврелиан', 'Автоном', 'Агап', 'Агапий', 'Агапит', 'Агафангел', 'Агафон', 'Аггей', 'Адам', 'Адриан', 'Азар', 'Азарий', 'Акакий', 'Акила', 'Аким', 'Акиндин', 'Акинф', 'Акинфий', 'Аксён', 'Аксентий', 'Александр', 'Алексей', 'Алексий', 'Альберт', 'Альфред', 'Амвросий', 'Амос', 'Амфилохий', 'Ананий', 'Анастасий', 'Анатолий', 'Андрей', 'Андриан', 'Андрон', 'Андроний', 'Андроник', 'Анект', 'Анемподист', 'Аникей', 'Аникий', 'Аникита', 'Анисий', 'Анисим', 'Антиох', 'Антип', 'Антипа', 'Антипий', 'Антон', 'Антонин', 'Антроп', 'Антропий', 'Ануфрий', 'Аполлинарий', 'Аполлон', 'Аполлос', 'Ардалион', 'Ардальон', 'Ареф', 'Арефий', 'Арий', 'Аристарх', 'Аристид', 'Аркадий', 'Арнольд', 'Арон', 'Арсений', 'Арсентий', 'Артамон', 'Артём', 'Артемий', 'Артур', 'Архип', 'Асаф', 'Асафий', 'Аскольд', 'Афанасий', 'Афиноген', 'Афинодор', 'Африкан', 'Бажен', 'Бенедикт', 'Богдан', 'Болеслав', 'Бонифат', 'Бонифатий', 'Борис', 'Борислав', 'Бронислав', 'Будимир', 'Вавила', 'Вадим', 'Валентин', 'Валериан', 'Валерий', 'Варлам', 'Варламий', 'Варнава', 'Варсоноф', 'Варсонофий', 'Варфоломей', 'Василий', 'Вассиан', 'Велизар', 'Велимир', 'Венедикт', 'Вениамин', 'Венцеслав', 'Веньямин', 'Викентий', 'Виктор', 'Викторий', 'Викул', 'Викула', 'Вилен', 'Виленин', 'Вильгельм', 'Виссарион', 'Вит', 'Виталий', 'Витовт', 'Витольд', 'Владилен', 'Владимир', 'Владислав', 'Владлен', 'Влас', 'Власий', 'Вонифат', 'Вонифатий', 'Всеволод', 'Всеслав', 'Вукол', 'Вышеслав', 'Вячеслав', 'Гавриил', 'Гаврил', 'Гаврила', 'Галактион', 'Гедеон', 'Гедимин', 'Геласий', 'Гелий', 'Геннадий', 'Генрих', 'Георгий', 'Герасим', 'Гервасий', 'Герман', 'Гермоген', 'Геронтий', 'Гиацинт', 'Глеб', 'Гораций', 'Горгоний', 'Гордей', 'Гостомысл', 'Гремислав', 'Григорий', 'Гурий', 'Гурьян', 'Давид', 'Давыд', 'Далмат', 'Даниил', 'Данил', 'Данила', 'Дементий', 'Демид', 'Демьян', 'Денис', 'Денисий', 'Димитрий', 'Диомид', 'Дионисий', 'Дмитрий', 'Добромысл', 'Добрыня', 'Довмонт', 'Доминик', 'Донат', 'Доримедонт', 'Дормедонт', 'Дормидбнт', 'Дорофей', 'Досифей', 'Евгений', 'Евграф', 'Евграфий', 'Евдоким', 'Евлампий', 'Евлогий', 'Евмен', 'Евмений', 'Евсей', 'Евстафий', 'Евстахий', 'Евстигней', 'Евстрат', 'Евстратий', 'Евтихий', 'Евфимий', 'Егор', 'Егорий', 'Елизар', 'Елисей', 'Елистрат', 'Елпидифор', 'Емельян', 'Епифан', 'Епифаний', 'Еремей', 'Ермий', 'Ермил', 'Ермила', 'Ермилий', 'Ермолай', 'Ерофей', 'Ефим', 'Ефимий', 'Ефрем', 'Ефремий', 'Захар', 'Захарий', 'Зенон', 'Зиновий', 'Зосим', 'Зосима', 'Иаким', 'Иакинф', 'Иван', 'Игнат', 'Игнатий', 'Игорь', 'Иероним', 'Измаил', 'Измарагд', 'Изосим', 'Изот', 'Изяслав', 'Илларион', 'Илиодор', 'Илья', 'Иннокентий', 'Иоанн', 'Йов', 'Иона', 'Иосафат', 'Иосиф', 'Ипат', 'Ипатий', 'Ипполит', 'Ираклий', 'Иринарх', 'Ириней', 'Иродион', 'Исаак', 'Исаакин', 'Исай', 'Исак', 'Исакий', 'Исидор', 'Иустин', 'Казимир', 'Каллимах', 'Каллиник', 'Каллиопий', 'Каллист', 'Каллистрат', 'Каллисфен', 'Калуф', 'Кандидий', 'Кантидиан', 'Капик', 'Капитон', 'Карион', 'Карл', 'Карп', 'Кастрихий', 'Касьян ', 'Ким', 'Киприан', 'Кир', 'Кириак', 'Кирик', 'Кирилл', 'Кирсан', 'Клавдий', 'Клим', 'Климент', 'Климентий', 'Кондрат', 'Кондратий', 'Конон', 'Конрад', 'Константин', 'Корней', 'Корнелий', 'Корнил', 'Корнилий', 'Ксенофонт', 'Кузьма', 'Куприян', 'Лавр', 'Лаврентий', 'Ладимир', 'Лазарь', 'Ларион', 'Лев', 'Леон', 'Леонард', 'Леонид', 'Леонтий', 'Леопольд', 'Логвин', 'Лонгин', 'Лука', 'Лукан', 'Лукьян', 'Любим', 'Любомир', 'Любомысл', 'Люциан', 'Мавр', 'Маврикий', 'Мавродий', 'Май', 'Макар', 'Макарий', 'Македон', 'Македоний', 'Максим', 'Максимиан', 'Максимилиан', 'Малх', 'Мануил', 'Марат', 'Мардарий', 'Мариан', 'Марин', 'Марк', 'Маркел', 'Маркиан', 'Марлен', 'Мартимьян', 'Мартин', 'Мартиниан', 'Мартирий', 'Мартин', 'Мартьян', 'Матвей', 'Мелентий', 'Мелетий', 'Меркул', 'Меркурий', 'Мефодий', 'Мечислав', 'Милан', 'Милен', 'Милий', 'Мина', 'Минай', 'Мирон', 'Мирослав', 'Мисаил', 'Митрофан', 'Митрофаний', 'Михаил', 'Михей', 'Модест', 'Моисей', 'Мокей', 'Мокий', 'Мстислав', 'Назар', 'Назарий', 'Наркис', 'Натан', 'Наум', 'Нестер', 'Нестор', 'Нефёд', 'Никандр', 'Никанор', 'Никита', 'Никифор', 'Никодим', 'Николай', 'Никон', 'Нил', 'Нифонт', 'Олег', 'Олимпий', 'Онисим', 'Онисифор', 'Онуфрий', 'Орест', 'Осип', 'Оскар', 'Остап', 'Остромир', 'Павел', 'Павлин', 'Паисий', 'Палладий', 'Памфил', 'Памфилий', 'Панкрат', 'Панкратий', 'Пантелей', 'Пантелеймон', 'Панфил', 'Парамон', 'Пармен', 'Парфён', 'Парфений', 'Парфентий', 'Патрикей', 'Патрикий', 'Пафнутий', 'Пахом', 'Пахомий', 'Перфилий', 'Пётр', 'Пимен', 'Питирим', 'Платон', 'Полиевкт', 'Полиект', 'Поликарп', 'Поликарпий', 'Порфир', 'Порфирий', 'Потап', 'Потапий', 'Пров', 'Прокл', 'Прокоп', 'Прокопий', 'Прокофий', 'Протас', 'Протасий', 'Прохор', 'Радий', 'Радим', 'Радислав', 'Радован', 'Ратибор ', 'Ратмир', 'Рафаил', 'Роберт', 'Родион', 'Роман', 'Ростислав', 'Рубен', 'Рувим', 'Рудольф', 'Руслан', 'Рюрик', 'Савва', 'Савватей', 'Савватий', 'Савёл', 'Савелий', 'Саверий', 'Савин ', 'Савиниан', 'Сакердон', 'Салтан', 'Самбила', 'Самсон', 'Самсоний', 'Самуил', 'Светозар', 'Свирид', 'Святополк', 'Святослав', 'Себастьян', 'Севастьян', 'Северин', 'Северьян', 'Селиван', 'Селивёрст', 'Селифан', 'Семён', 'Семион', 'Серапион', 'Серафим', 'Сергей', 'Сигизмунд', 'Сидор', 'Сила', 'Силан', 'Силантий', 'Силуян', 'Сильван', 'Сильвестр', 'Симон', 'Смарагд', 'Созон', 'Созонт', 'Созонтий', 'Сократ', 'Соломон', 'Сосипатр', 'Софон', 'Софоний', 'Софрон', 'Софроний', 'Спартак', 'Спиридон', 'Спиридоний', 'Станимир', 'Стахий', 'Станислав', 'Степан', 'Стоян', 'Стратбник', 'Сысой', 'Тарас', 'Твердислав', 'Творимир', 'Терентий', 'Тертий', 'Тигран', 'Тигрий', 'Тимофей', 'Тимур', 'Тит', 'Тихон', 'Тристан', 'Трифилий', 'Трифон', 'Трофим', 'Увар', 'Ульян', 'Устин', 'Фабиан', 'Фадей', 'Фалалей', 'Фатьян', 'Фёдор', 'Федос', 'Федосей', 'Федосий', 'Федот', 'Федотий', 'Федул', 'Феликс', 'Фемистокл', 'Феогност', 'Феоктист', 'Феофан', 'Феофил', 'Феофилакт', 'Ферапонт', 'Филарет', 'Филат', 'Филимон', 'Филипий', 'Филипп', 'Филофей', 'Фирс', 'Флавиан', 'Флавий', 'Флегонт', 'Флорентий', 'Флорентин', 'Флориан', 'Фока', 'Фома', 'Фортунат', 'Фотий', 'Фридрих', 'Фрол', 'Харитон', 'Харитоний', 'Харлам', 'Харламп', 'Харлампий', 'Хрисанф', 'Христофор', 'Эдуард', 'Эмилий', 'Эмиль', 'Эммануил', 'Эразм', 'Эраст', 'Эрнест', 'Эрнст', 'Ювеналий', 'Юлиан', 'Юлий', 'Юрий', 'Юстиниан', 'Яким', 'Яков', 'Якуб', 'Ян', 'Януарий', 'Ярополк', 'Ярослав']

In [15]:
stops_names_w_low = []
stops_names_m_low = []

In [16]:
for el in stops_names_w:
    stops_names_w_low.append(el.lower())
    
for el_m in stops_names_m:
    stops_names_m_low.append(el.lower())

In [17]:
stops_names_w_low[1:4]

['августина', 'авдотья', 'аврелия']

In [18]:
stops_new = ['что', 'это', 'как', 'бесплтно', 'бесплатно', 'безкоштовно', 'даром', 'free', 'бесплтно', 'руками', 'самостоятельно', 'самодельный', 'скачать', 'download', 'драйвер', 'прошивка', 'порно', 'порнушка', 'вконтакте', 'доклад', 'porn', 'porno', 'sex', 'форум', 'реферат', 'статья', 'блог', 'хитрости', 'уловки', 'хабр', 'хабра', 'хабрхабр', 'faq', 'инструкция', 'википедия', 'вики', 'wikipedia', 'wiki', 'схема', 'состав', 'способ', 'методы', 'технология', 'торрент', 'торент', 'torrent', 'torent', 'torents', 'torrents', 'сонник', 'приснилось', 'прикол', 'смешно', 'юмор', 'анекдот', 'эротика', 'эротический', 'аамир', 'аариф', 'аарон', 'аасим', 'абан', 'аббас', 'абд', 'абдула', 'абдулла', 'абдульазиз', 'абдульазым', 'абдульали', 'абдульалим', 'абдульбари', 'абдульбасыт', 'абдульвадуд', 'абдульвахаб', 'абдульвахид', 'абдульгафар', 'абдульгафур', 'абдульджабар', 'абдульджалиль', 'абдулькадир', 'абдулькарим', 'абдулькахар', 'абдулькудус', 'абдульлатыф', 'абдульмаджид', 'абдульматин', 'абдульмуджиб', 'абдульмуиз', 'абдульмуталь', 'абдульмухаймин', 'абдульфатах', 'абдульхади', 'абдульхакам', 'абдульхаким', 'абдульхакк', 'абдульхалик', 'абдульхалим', 'абдульхамид', 'абдульхасиб', 'абдульхафиз', 'абдуннасер', 'абдурразак', 'абдуррауф', 'абдуррафи', 'абдуррахим', 'абдуррахман', 'абдуррашид', 'абдуссабур', 'абдуссалям', 'абдуссамад', 'абдуссами', 'абдутавваб', 'абдущщакур', 'абидин', 'або', 'абрам', 'абраха', 'абу', 'абульхайр', 'ава', 'авад', 'авангард', 'авва', 'аввакум', 'авгей', 'авгий', 'август', 'августин', 'авда', 'авдей', 'авделай', 'авдиес', 'авдий', 'авдикий', 'авдифакс', 'авдон', 'авенир', 'авентин', 'аверкий', 'аверьян', 'авив', 'авид', 'авим', 'авксентий', 'авкт', 'авраам', 'аврам', 'авраша', 'аврелий', 'аврор', 'авсей', 'автолик', 'автомедон', 'автоном', 'авудим', 'авундий', 'агав', 'агавва', 'агамемнон', 'аганя', 'агап', 'агапий', 'агапит', 'агапон', 'агафангел', 'агафодор', 'агафон', 'агафоник', 'агафоника', 'агафопод', 'агафопус', 'агаша', 'аггей', 'агей', 'агелай', 'агенор', 'агзам', 'аглаий', 'агн', 'агрий', 'адад', 'адам', 'адиб', 'аднан', 'адольф', 'адонис', 'адриан', 'адыль', 'аетий', 'азадан', 'азам', 'азамат', 'азар', 'азарий', 'азат', 'аззам', 'азиз', 'азий', 'азхар', 'аид', 'аифал', 'айдар', 'айет', 'айман', 'айнур', 'айрат', 'айтуган', 'айюб', 'акакий', 'акаша', 'акепсий', 'акепсим', 'акила', 'аким', 'акиндин', 'акинфий', 'акрам', 'аксен', 'акустий', 'акутион', 'алан', 'алар', 'алвиан', 'алевтин', 'алей', 'александр', 'алексей', 'алексий', 'ален', 'алет', 'алехандро', 'алеша', 'али', 'алим', 'алимпий', 'алипий', 'алканой', 'алкимедон', 'алкиной', 'алкмеон', 'аллар', 'алмас', 'алон', 'алфей', 'альаббас', 'альберт', 'альбин', 'альварес', 'альфонс', 'альфред', 'аляуддин', 'амадей', 'амвросий', 'амджад', 'амин', 'амир', 'аммар', 'аммон', 'аммун', 'амон', 'амонит', 'амос', 'амплий', 'амр', 'амфиан', 'амфилохий', 'амфион', 'ананда', 'ананий', 'анас', 'анастас', 'анастасий', 'анатолий', 'анатом', 'анвар', 'ангел', 'ангелар', 'ангий', 'андрей', 'андремон', 'андриан', 'андромаха', 'андрон', 'андроник', 'анект', 'анемподист', 'аний', 'аникита', 'анимаиса', 'анин', 'анис', 'анисим', 'анри', 'ансар', 'антей', 'антенор', 'антилох', 'антимах', 'антиной', 'антиох', 'антип', 'антипа', 'антипатр', 'антиф', 'антифат', 'антон', 'антоний', 'антонин', 'анту', 'антуан', 'ану', 'анубис', 'анувий', 'анурий', 'ануфрий', 'анфим', 'анфимий', 'анфипом', 'анфир', 'анфуса', 'анхиал', 'анхиз', 'апеллий', 'апизаон', 'апис', 'аполл', 'аполлинарий', 'аполлон', 'аполлоний', 'аполлос', 'апрониан', 'апфимедон', 'аргент', 'ардалион', 'арджуна', 'арей', 'аретос', 'арефа', 'арефий', 'ариадна', 'ариан', 'арий', 'арис', 'аристарх', 'аристид', 'аристион', 'аристовул', 'аристоклий', 'аркадий', 'аркисий', 'арнеон', 'арнольд', 'аронос', 'арпила', 'аррунс', 'арсен', 'арсений', 'арсентий', 'арслан', 'артамон', 'артем', 'артема', 'артемий', 'артемон', 'артур', 'аруру', 'архелий', 'архептолем', 'архип', 'архипп', 'арчил', 'арш', 'арэфа', 'асад', 'асадулла', 'асан', 'асинкрит', 'асиф', 'аскалаф', 'аскалон', 'асканий', 'асклепий', 'асклипиодот', 'аскольд', 'аслан', 'асма', 'ассарак', 'астафий', 'астерий', 'астий', 'астион', 'астионакс', 'астрид', 'ата', 'атаа', 'атеист', 'атон', 'атрей', 'аттий', 'аттик', 'атэ', 'аус', 'афанасий', 'афзал', 'афиноген', 'афинодор', 'афиф', 'афон', 'афоний', 'афоня', 'африкан', 'ахад', 'ахаз', 'ахаик', 'ахат', 'ахбар', 'ахила', 'ахилл', 'ахиллий', 'ахмад', 'ахмар', 'ахмат', 'ахунд', 'ацест', 'ашраф', 'аяз', 'аякс', 'баасым', 'бадруддин', 'бажен', 'бакир', 'балаш', 'бальтазар', 'барат', 'бартоломей', 'басам', 'басыль', 'батист', 'баха', 'бахауддин', 'бахир', 'бахиуддин', 'бахтияр', 'башаар', 'баяз', 'баян', 'бектуган', 'бел', 'белит сери', 'беллерофонт', 'бенедикт', 'берислав', 'бернард', 'бертольд', 'бертрам', 'бес', 'бианер', 'бидзин', 'бикбай', 'бикбулат', 'билал', 'билалль', 'битий', 'бишр', 'благомир', 'боб', 'богдан', 'боголеп', 'боеслав', 'болеслав', 'бонифатий', 'борей', 'боримир', 'борис', 'борислав', 'боян', 'бронислав', 'будимир', 'булат', 'буранбай', 'бурангул', 'буранша', 'бурхан', 'вавила', 'ваджих', 'вади', 'вадим', 'вазих', 'ваиль', 'вакил', 'вакх', 'валент', 'валентин', 'валера', 'валериан', 'валерий', 'вали', 'валид', 'валиуддин', 'вальтер', 'ваптос', 'варадат', 'варахий', 'варахисий', 'варвар', 'варипсав', 'варлаам', 'варлам', 'варнава', 'варсава', 'варсис', 'варсонофий', 'варул', 'варух', 'варфоломей', 'васил', 'василек', 'василид', 'василий', 'василиск', 'василько', 'васим', 'васой', 'васс', 'вассиан', 'вася', 'вафик', 'вафусий', 'вахид', 'вахрамей', 'вахтисий', 'вацлав', 'велимир', 'велислав', 'велор', 'вендимиан', 'венедикт', 'венедим', 'вениамин', 'вергилий', 'верк', 'вечеслав', 'вианор', 'викарий', 'викентий', 'виктор', 'викторин', 'вил', 'вилен', 'вилли', 'вильгельм', 'вильдан', 'вильям', 'винсент', 'вир', 'вирилад', 'виссарион', 'вит', 'виталий', 'витольд', 'витя', 'вифоний', 'влад', 'владилен', 'владимир', 'владислав', 'владлен', 'влас', 'власий', 'вова', 'воин', 'воислав', 'володар', 'волсцент', 'вольдемар', 'вольмир', 'вольф', 'вольфганг', 'воля', 'вонифатий', 'всеволод', 'всемил', 'вук', 'вукол', 'вулкан', 'вылко', 'вячеслав', 'габдулла', 'габит', 'габриель', 'гавведай', 'гавиний', 'гавриил', 'гаврила', 'гази', 'газиз', 'газим', 'гаий', 'гай', 'галактион', 'галиб', 'галик', 'галим', 'галионт', 'галиферс', 'гамалиил', 'гани', 'ганимед', 'гантиол', 'гапон', 'гарольд', 'гарри', 'гаррик', 'гасан', 'гастон', 'гата', 'гафар', 'гаян', 'гед', 'гектор', 'геласий', 'гелен', 'гелиан', 'гелий', 'геликаон', 'гелиос', 'гемелл', 'гена', 'геннадий', 'генрих', 'георгий', 'геракл', 'геральд', 'герасим', 'герберт', 'гервасий', 'геркулес', 'герман', 'германн', 'гермес', 'гермоген', 'геронтий', 'гертруд', 'гершель', 'гетион', 'гефест', 'гигантий', 'гильберт', 'гиппокоон', 'гипполох', 'гиппот', 'гиппофой', 'гипсенор', 'гиртак', 'гияс', 'главк', 'глеб', 'гликерий', 'гоброн', 'гор', 'горазд', 'горгий', 'горгоний', 'гордей', 'гордиан', 'гордий', 'горимир', 'горислав', 'горн', 'горус', 'гранит', 'грегор', 'григорий', 'гур', 'гурий', 'гурьян', 'густав', 'давид', 'давикт', 'давлет', 'давуд', 'давыд', 'дада', 'далмат', 'дамас', 'дамиан', 'дамир', 'дан', 'данай', 'данакт', 'даниил', 'данила', 'дар', 'дардан', 'дарес', 'дарий', 'дассий', 'дауд', 'даян', 'деифоб', 'декабрий', 'дементий', 'демид', 'демодок', 'демокоон', 'демократ', 'демосфен', 'демьян', 'дениз', 'денис', 'джабир', 'джавад', 'джалаль', 'джамаль', 'джамальуддин', 'джамиль', 'джафар', 'джек', 'джеральд', 'джим', 'джихад', 'джо', 'джозеф', 'джон', 'джордж', 'дидим', 'дидумос', 'дий', 'дик', 'дима', 'димант', 'димитриан', 'димитрий', 'диодор', 'диодот', 'диоклес', 'диомед', 'диомид', 'дион', 'дионисий', 'диоскор', 'дисан', 'дисидерий', 'дитрих', 'дияуддин', 'дмитрий', 'дмитро', 'добрыня', 'довмонт', 'долий', 'долион', 'долон', 'дометиан', 'дометий', 'домн', 'домнин', 'дональд', 'дональт', 'донат', 'доримедонт', 'дормидонт', 'дорофей', 'досифей', 'драгутин', 'дукитий', 'дула', 'евагрий', 'евангел', 'еварест', 'еввентий', 'еввул', 'евгений', 'евграф', 'евдоким', 'евдоксий', 'евелпист', 'евеопист', 'евиласий', 'евкарпий', 'евклий', 'евкул', 'евлампий', 'евлогий', 'евмений', 'евникиан', 'евноик', 'евод', 'евпл', 'евпор', 'евпсихий', 'евсевий', 'евсевон', 'евсей', 'евсигний', 'евстафий', 'евстахий', 'евстигней', 'евстохий', 'евстратий', 'евсхимон', 'евтихиан', 'евтихий', 'евтропий', 'евфимиан', 'евфимий', 'евфрасий', 'евфросин', 'егор', 'едесий', 'екдит', 'ексакустодиан', 'елеазар', 'елевсипп', 'елевферий', 'елезвой', 'елизар', 'елизбар', 'елисей', 'елистрат', 'елладий', 'еллий', 'елпидий', 'елпидифор', 'емелиан', 'емельян', 'емеля', 'емилиан', 'енелпист', 'епафрас', 'епафродит', 'епенет', 'епикнет', 'епиктет', 'епимах', 'епифан', 'епифаний', 'епполоний', 'еразм', 'ераст', 'ерема', 'еремей', 'ерм', 'ермак', 'ермий', 'ермил', 'ерминингельд', 'ермипп', 'ермоген', 'ермократ', 'ермолай', 'ерос', 'ерофей', 'еруслан', 'еспер', 'еферий', 'ефив', 'ефим', 'ефимий', 'ефрем', 'жак', 'жан', 'ждан', 'жозеф', 'жорж', 'жюль', 'жюльен', 'зайд', 'закарийя', 'закария', 'заки', 'закир', 'закхей', 'замам', 'зафир', 'захар', 'захария', 'захид', 'захир', 'зевин', 'зевс', 'зенон', 'зефир', 'зигмунд', 'зигфрид', 'зиннур', 'зиновий', 'зинон', 'зияд', 'зоил', 'зорий', 'зосим', 'зосима', 'зот', 'зотик', 'зульфикар', 'зухайр', 'иадор', 'иакинф', 'иакисхол', 'иаков', 'иамвлих', 'ианикит', 'ианнуарий', 'иасон', 'ибрагим', 'ибрахим', 'иван', 'игафракс', 'игнатий', 'игорь', 'идей', 'идоменей', 'идрис', 'иезекииль', 'иеракс', 'иеремия', 'иерон', 'иероним', 'иерофей', 'из', 'иззуддин', 'измаил', 'изяслав', 'иисус', 'ий', 'икар', 'икарий', 'икрам', 'ил', 'иларий', 'иларион', 'илиан', 'илиодор', 'илионей', 'илларион', 'иллирик', 'ильгиз', 'ильдар', 'ильдус', 'ильшат', 'илья', 'имад', 'имадуддин', 'иман', 'имил', 'иммануил', 'имран', 'имхатеп', 'инвикт', 'индис', 'иней', 'иннокентий', 'инсар', 'инсаф', 'иоад', 'иоаким', 'иоанн', 'иоанникий', 'иоасаф', 'иов', 'иоганн', 'иоиль', 'иосиф', 'ипат',
             'ипатий', 'иперехий', 'ипполит', 'иракламвон', 'ираклемон', 'ираклий', 'ирек', 'иринарх', 'ириней', 'ириний', 'ирнини', 'иродион', 'ирфан', 'иса', 'исаак', 'исаакий', 'исавр', 'исаия', 'исай', 'исак', 'исам', 'исе', 'исидор', 'исихий', 'искандер', 'иской', 'ислам', 'исмагил', 'исмаил', 'исмат', 'истукарий', 'исфандияр', 'исхак', 'иттифак', 'иувеналий', 'иувентин', 'иувиналий', 'иуда', 'иулиан', 'иулий', 'иуст', 'иустин', 'иустиниан', 'ифидамас', 'ификл', 'ифит', 'ихи', 'ихсан', 'ишбулат', 'ишбулды', 'ишгильды', 'иштуган', 'ишуллану', 'июлий', 'кабир', 'кадм', 'кадыр', 'казимир', 'кайс', 'каленик', 'калигула', 'калиник', 'калистрат', 'каллимах', 'каллиник', 'каллиопий', 'каллист', 'каллистрат', 'калуф', 'калхас', 'камал', 'камиль', 'кандид', 'каниз', 'кантидиан', 'кантидий', 'капаней', 'капик', 'капис', 'капитон', 'карим', 'карион', 'карл', 'карп', 'картерий', 'касим', 'кассиан', 'кастор', 'кастрикий', 'кастул', 'касым', 'касьян', 'катерий', 'катилл', 'катун', 'кашфулла', 'каюм', 'квинтилиан', 'кебрен', 'кебрион', 'келестин', 'келсий', 'кенсорин', 'кесарий', 'кесарь', 'кетевань', 'кеша', 'кикн', 'ким', 'киндей', 'кион', 'киприан', 'кир', 'кириак', 'кириан', 'кирик', 'кирилл', 'кирин', 'кирион', 'кирсан', 'кирьян', 'клавдиан', 'клавдий', 'клавз', 'клемент', 'клеоник', 'клим', 'климент', 'климентий', 'клитий', 'кодрат', 'коинт', 'колумбий', 'конан', 'кондрат', 'кондратий', 'конкордий', 'конон', 'конрад', 'константин', 'констанций', 'консуэло', 'коон', 'коприй', 'корас', 'кореб', 'корней', 'корнелий', 'корнил', 'корнилий', 'корнут', 'корон', 'коронат', 'косма', 'краснослав', 'крискент', 'крискентиан', 'крисп', 'кристиан', 'кристоф', 'кронид', 'кронион', 'ксантиппа', 'ксанфий', 'ксанфиппа', 'ксенофонт', 'ктезипп', 'ктесей', 'куарт', 'кудама', 'кузьма', 'кукша', 'куприян', 'курбан', 'курбат', 'кутайба', 'кутоний', 'куфий', 'лабиб', 'лавз', 'лавр', 'лаврентий', 'лазарь', 'лампад', 'ланселот', 'лаодам', 'лаодикий', 'лаодок', 'лаокоон', 'лаомедонт', 'ларгий', 'ларион', 'лариса', 'латин', 'лаэрт', 'леандр', 'лев', 'левк', 'левкий', 'лель', 'лениан', 'леня', 'леодей', 'леон', 'леонард', 'леонид', 'леонт', 'леонтей', 'леонтий', 'либерии', 'либерт', 'ливерий', 'ливий', 'ликаон', 'ликарион', 'ликомед', 'лимний', 'лин', 'лир', 'лисимах', 'лолий', 'лонгин', 'лот', 'луарсаб', 'луи', 'лука', 'лукиан', 'лукий', 'лукиллиан', 'лукман', 'лукьян', 'лупп', 'люаи', 'любим', 'любомир', 'любослав', 'людвиг', 'люксен', 'лютфи', 'маазин', 'маахир', 'мавр', 'маврикий', 'мавсима', 'магафур', 'магистриан', 'магн', 'магсум', 'маджит', 'мадлен', 'мажд', 'мажди', 'маждуддин', 'майрам', 'майсара', 'майслав', 'майсур', 'макар', 'макарий', 'македоний', 'макин', 'макровий', 'макс', 'максиан', 'максим', 'максимиан', 'максимилиан', 'максимиллиан', 'максимильян', 'максуд', 'максуз', 'малик', 'малх', 'мамант', 'мамдух', 'маммий', 'мамнун', 'мансур', 'мануил', 'манфред', 'мар', 'марван', 'мардарий', 'мардоний', 'марзук', 'мари', 'мариав', 'мариамна', 'марий', 'марин', 'марк', 'маркелл', 'маркеллин', 'маркиан', 'марлен', 'марон', 'марсалий', 'марсен', 'мартиан', 'мартин', 'мартиниан', 'мартирий', 'мартын', 'маруф', 'маснави', 'масуд', 'матвей', 'матфей', 'матфий', 'махаон', 'махди', 'махмуд', 'мегапенф', 'мегей', 'медимн', 'медон', 'мезенций', 'мелампод', 'мелантий', 'меласипп', 'мелевсипп', 'мелентий', 'мелетий', 'мелиссен', 'мелитон', 'мемнон', 'менандр', 'менелай', 'менесфей', 'менетий', 'менигн', 'ментес', 'ментор', 'мерген', 'мерион', 'меркурий', 'мертий', 'мессап', 'метиск', 'метопт', 'мефодий', 'мечислав', 'миан', 'мигдоний', 'мигшдон', 'мидхад', 'милад', 'милан', 'милен', 'милий', 'милонег', 'милослав', 'милютин', 'мин', 'миней', 'минеон', 'минос', 'минсифей', 'мир', 'миракс', 'мирон', 'мирослав', 'миртил', 'мисаил', 'мисбах', 'митра', 'митридат', 'митрофан', 'митя', 'мифтах', 'михаил', 'михей', 'мичура', 'модест', 'моисей', 'мокий', 'молосс', 'морон', 'мстислав', 'муаз', 'муайид', 'мувафак', 'муддарис', 'муджахид', 'муккарам', 'муко', 'мунзир', 'мунир', 'мунис', 'мунтасир', 'мурат', 'муртади', 'муса', 'муслим', 'мустафа', 'мут', 'мутаа', 'мутазз', 'мутасим', 'мути', 'муфид', 'мухаммад', 'мухаммед', 'муханнад', 'мухсин', 'мухтади', 'мухтар', 'наасим', 'наби', 'набиль', 'набих', 'набу', 'набхан', 'навплий', 'наджи', 'наджиб', 'наджмуддин', 'надим', 'надир', 'назар', 'назарий', 'назим', 'назир', 'назих', 'наиль', 'наим', 'намтар', 'нариман', 'наркис', 'наркисс', 'насим', 'насих', 'насреддин', 'нассеруддин', 'натан', 'наум', 'нафанаил', 'неделя', 'нейтх', 'нектарий', 'нелей', 'неон', 'неонил', 'неоптолем', 'неофит', 'нергал', 'нерей', 'нестор', 'нефмон', 'нефтис', 'низ', 'низам', 'никандр', 'никанор', 'никита', 'никифор', 'никодим', 'никола', 'николай', 'никон', 'никострат', 'никтополион', 'нил', 'ниниб', 'нисон', 'нит', 'нифонт', 'новомир', 'ной', 'норберт', 'норд', 'норман', 'нот', 'нуман', 'нуруддин', 'нух', 'овидий', 'одиссей', 'оилей', 'океан', 'октавиан', 'октябрин', 'октябрь', 'олаф', 'олег', 'олжас', 'оливер', 'олимп', 'олимпан', 'олимпий', 'ольгерд', 'омар', 'омврий', 'омейр', 'омран', 'оней', 'онисий', 'онисим', 'онисифор', 'оноре', 'онуфрий', 'опис', 'ор', 'орентий', 'орест', 'орион', 'орифаон', 'ормен', 'орменон', 'оронт', 'орфей', 'освальд', 'осип', 'осирис', 'оскар', 'остап', 'острихий', 'остромир', 'от', 'отто', 'оферионей', 'охрид', 'павлик', 'павлин', 'павсикакий', 'павсилип', 'паисий', 'паламед', 'палиас', 'палий', 'палладий', 'паллант', 'памва', 'памвон', 'памфалон', 'памфамир', 'памфил', 'пандар', 'панкратий', 'панлихарий', 'пансофий', 'пантелеймон', 'панфил', 'панфой', 'папа', 'папий', 'папила', 'парамон', 'пард', 'паригорий', 'парис', 'пармен', 'пармений', 'парод', 'партизан', 'парфений', 'пасикрат', 'патапий', 'патермуфий', 'патрикей', 'патрикий', 'патров', 'патрокл', 'пафнутий', 'пахом', 'пахомий', 'пелей', 'пелий', 'пелопс', 'пеон', 'перегрин', 'пересвет', 'перифас', 'перифой', 'персей', 'петей', 'петр', 'пигасий', 'пизаидр', 'пизандр', 'пизистрат', 'пий', 'пилад', 'пилон', 'пимен', 'пинна', 'пионий', 'пирей', 'пирехм', 'пирр', 'пист', 'питирим', 'платон', 'плисфен', 'плутон', 'подарк', 'пой', 'полиб', 'полидам', 'полидевк', 'полидор', 'полиевкт', 'полиен', 'поликарп', 'полиместор', 'полиник', 'полинур', 'полипетю', 'полит', 'полифем', 'полихроний', 'полувий', 'помпеи', 'помпей', 'помпий', 'понтий', 'поплии', 'поплий', 'порфирий', 'посейдон', 'потамий', 'потап', 'потит', 'приам', 'прилидиан', 'принкипс', 'приск', 'провий', 'прозор', 'прокесс', 'прокл', 'прокоп', 'прокопий', 'прокофий', 'прокул', 'протас', 'протасий', 'протезилай', 'протей', 'протерий', 'протоген', 'протолеон', 'профан', 'прохор', 'псой', 'птах', 'пуд', 'пузур бел', 'пуплий', 'ра', 'рабах', 'раби', 'равиль', 'рагиб', 'раджаб', 'радий', 'радим', 'радимир', 'радислав', 'радомир', 'радослав', 'ражден', 'разумник', 'раид', 'ракин', 'рамаз', 'рамиз', 'рамиль', 'расиль', 'расим', 'расих', 'рассул', 'ратиб', 'ратибор', 'ратмир', 'рауф', 'рафаил', 'рафи', 'рафик', 'рахим', 'рахман', 'рашад', 'рашид', 'реас', 'ревмир', 'ревокат', 'рез', 'рексенор', 'рем', 'ренат', 'ридван', 'ризван', 'рикс', 'ричард', 'рияд', 'роберт', 'родерик', 'родион', 'родопиан', 'ролан', 'роланд', 'роман', 'ромил', 'ромул', 'ростислав', 'рубен', 'рудольф', 'руслан', 'рустам', 'рустик', 'руф', 'руфин', 'рушан', 'рэм', 'рюрик', 'саад', 'сабир', 'сабит', 'сабих', 'савва', 'савватий', 'савел', 'савелий', 'саверий', 'савин', 'саджид', 'садок', 'садоф', 'садри', 'садык', 'сазон', 'саид', 'сайид', 'сайфи', 'сайфуддин', 'сакердон', 'сакиб', 'салават', 'саламан', 'салах', 'салахуддин', 'салем', 'салим', 'салих', 'салман', 'салмоней', 'сальвадор', 'самад', 'самеи', 'самиир', 'самир', 'самон', 'сампсон', 'самсон', 'самуил', 'саня', 'сарвил', 'сарпедон', 'сасоний', 'сатир', 'саторин', 'саторний', 'саторнил', 'саторнин', 'саттар', 'сатурн', 'сатурнин', 'саул', 'сафван', 'сафи', 'сахир', 'саша', 'свет', 'светлан', 'светозар', 'светослав', 'святогор', 'святополк', 'святослав', 'севастиан', 'севастьян', 'север', 'севериан', 'северин', 'северьян', 'севир', 'секунд', 'селевк', 'селевкий', 'селиний', 'селькис', 'семен', 'сеннис', 'сеня', 'серапион', 'серапис', 'серафим', 'сергей', 'сергий', 'сет', 'сивел', 'сигиц', 'сидор', 'сизиф', 'сикст', 'сила', 'силан', 'силантий', 'силуан', 'сильван', 'сильвест', 'сильвестр', 'симбир', 'симеон', 'симоисий', 'симон', 'симфориан', 'син', 'синон', 'сионий', 'сирадж', 'сисиний', 'сисой', 'слава', 'славомир', 'смарагд', 'собек', 'совет', 'созонт', 'сок', 'сократ', 'соломон', 'солохон', 'сонирил', 'сосий', 'сосипатр', 'сосс', 'соссий', 'сосфен', 'софрон', 'софроний', 'спартак', 'спевсипп', 'сперхий', 'спиридон', 'сталь', 'станислав', 'стахий', 'стентор', 'степан', 'стефан', 'стратион', 'стратон', 'стратоник', 'стратор', 'строфий', 'студий', 'субхи', 'суимвл', 'сулейман', 'султан', 'суляйман', 'сумукан', 'сууд', 'сухайб', 'сухайль', 'сухий', 'сфенел', 'сысой', 'таамир', 'таахир', 'таврион', 'тагир', 'таир', 'таймулла', 'тайсир', 'талаль', 'таламон', 'талха', 'тальфибий', 'тамам', 'таммуз', 'тансылу', 'тантал', 'тарас', 'тарасий', 'тарах', 'тарик', 'тариф', 'таричан', 'тарфах', 'татион', 'тауфик', 'тахир', 'тевер', 'тевкр', 'тезей', 'теймураз', 'телем', 'телемах', 'телеф', 'теодор', 'терентий', 'тертий', 'тивуртий', 'тигран', 'тигрий', 'тидей', 'тимолай', 'тимон', 'тимофей', 'тимур', 'тиндарей', 'тиррей', 'тирсей', 'тит', 'титий', 'тифон', 'тихик', 'тихомир', 'тихон', 'тлеполем', 'толя', 'том', 'томас', 'тоос', 'тот', 'тоша', 'транквиллин', 'трерис', 'трибун', 'тривимий', 'тристан', 'тритон', 'трифиллий', 'трифон', 'троадий', 'троил', 'трой', 'трофим', 'турвон', 'турн', 'туссэн', 'уалент', 'уалентин', 'уалерий', 'уалириан', 'уар', 'убада', 'убай', 'убайда', 'убара туму', 'уврикий', 'узбек', 'укба', 'ульмас', 'ульрих', 'умайр', 'умар', 'упуаут', 'ур эа', 'урал', 'урван', 'уриил', 'урош', 'урпасиан', 'урус', 'усама', 'усман', 'устин', 'ут напиштим', 'утба', 'фавий', 'фавмасий', 'фавн', 'фавст', 'фавстиан', 'фавуаз', 'фаддей', 'фадей', 'фади', 'фадль', 'фадлюллах', 'фаиз', 
    'фаик', 'файзулла', 'файсал', 'фал', 'фалалей', 'фалассий', 'фаллелей', 'фанис', 'фантин', 'фарид', 'фарис', 'фармуфий', 'фарнакий', 'фарук', 'фархад', 'фатин', 'фатих', 'фауст', 'фахад', 'фахир', 'фахри', 'фаяз', 'феб', 'февралин', 'фегей', 'федим', 'федон', 'федор', 'федос', 'федосей', 'федосий', 'федот', 'федотий', 'федул', 'феклист', 'феликс', 'фемий', 'фемистокл', 'фемистоклей', 'феникс', 'фенникс', 'феоген', 'феогнид', 'феогний', 'феогност', 'феодор', 'феодорит', 'феодосий', 'феодот', 'феодох', 'феодул', 'феоклимен', 'феоктирист', 'феоктист', 'феопемпт', 'феопист', 'феопрепий', 'феост', 'феостирикт', 'феостих', 'феотекн', 'феотих', 'феофан', 'феофил', 'феофилакт', 'ферапонт', 'фердинанд', 'ферм', 'ферсандр', 'ферсит', 'феспесий', 'феспорт', 'фидель', 'филагрий', 'филадельф', 'филарет', 'филат', 'филей', 'филетер', 'филикиссим', 'филикс', 'филимон', 'филипп', 'филит', 'филл', 'филогоний', 'филоктeт', 'филоктимон', 'филолог', 'филон', 'философ', 'филотий', 'филофей', 'филумен', 'фимэт', 'финеес', 'финей', 'фирмин', 'фирмос', 'фирс', 'фифаил', 'флавиан', 'флавий', 'флегонт', 'флор', 'флорентий', 'флоренц', 'флорин', 'фоас', 'фока', 'фома', 'фоон', 'форвин', 'фортунат', 'фостирий', 'фот', 'фотий', 'фотин', 'франк', 'франц', 'франциск', 'фрасимед', 'фридрих', 'фрол', 'фроний', 'фронтасий', 'фронтис', 'фрументий', 'фуад', 'фулвиан', 'фусик', 'хаашим', 'хабиб', 'хабибулла', 'хади', 'хадис', 'хаим', 'хайдар', 'хайри', 'хайруддин', 'хайсам', 'хаким', 'халдун', 'халид', 'халил', 'халилль', 'хамза', 'хамид', 'хани', 'харалампий', 'харис', 'харисим', 'харитон', 'харлампий', 'харон', 'харун', 'хасан', 'хассан', 'хатим', 'хатор', 'хафиз', 'хашим', 'херимон', 'хикмет', 'хирон', 'хнум', 'ходжа', 'хонс', 'храбр', 'хрис', 'хрисанта', 'хрисанф', 'хрисипп', 'хрисогон', 'хрисотель', 'христиан', 'христофор', 'худ', 'худион', 'хузайма', 'хузайфа', 'хумам', 'хумбаба', 'хусам', 'хусамуддин', 'хусдазат', 'хуссейн', 'цезарь', 'цекул', 'цетон', 'часлав', 'чеслав', 'чингиз', 'шади', 'шакир', 'шалва', 'шамаш', 'шамиль', 'шамси', 'шариф', 'шафик', 'шахрияр', 'шепре', 'шерлок', 'шимшон', 'шио', 'шихаб', 'шура', 'эа', 'эварндр', 'эвдор', 'эвемон', 'эвмед', 'эвмел', 'эвриал', 'эврибат', 'эвридам', 'эврилох', 'эвримах', 'эврином', 'эврипил', 'эврисак', 'эврисфей', 'эврит', 'эвфорб', 'эгипс', 'эгиптий', 'эгисф', 'эгмонт', 'эдвард', 'эдик', 'эдип', 'эдит', 'эдуард', 'электрон', 'элизбар', 'элой', 'элопей', 'эльбрус', 'эльдар', 'эльпепор', 'эмели', 'эмилий', 'эмиль', 'эммануил', 'энгель', 'эней', 'энергий', 'эниопей', 'эномай', 'эол', 'эос', 'эпей', 'эпикл', 'эразм', 'эраст', 'эрик', 'эрихтопий', 'эрнест', 'эрнст', 'эрот', 'эсфирь', 'этеоней', 'эфиальт', 'эфопс', 'эфра', 'эхеней', 'эхепел', 'эхет', 'эхефрон', 'ювеналий', 'ювентин', 'ювиналий', 'юджин', 'юлдаш', 'юлиан', 'юлий', 'юнус', 'юпитер', 'юра', 'юрий', 'юст', 'юстин', 'юстиниан', 'юсуф', 'ютурна', 'юхан', 'ядгар', 'яков', 'якуб', 'якут', 'ямал', 'яман', 'ян', 'яннуарий', 'януарий', 'янус', 'яромир', 'ярополк', 'ярослав', 'ясар', 'ясий', 'ясин', 'ясон', 'яхья', 'аалия', 'аамаль', 'аасма', 'абир', 'аблаа', 'августа', 'авгэ', 'авдотья', 'аврелия', 'аврора', 'агапия', 'агата', 'агафия', 'агафоклия', 'агафья', 'аглаида', 'аглая', 'агнеса', 'агнесса', 'агния', 'аграфена', 'агриппина', 'ада', 'адаб', 'аделаида', 'аделина', 'аделия', 'адель', 'адиля', 'адриана', 'аеропа', 'аза', 'азалия', 'азария', 'азза', 'азиза', 'аида', 'айгуль', 'айда', 'айех', 'айша', 'айя', 'акилина', 'аксинья', 'акулина', 'алевтина', 'александра', 'алена', 'алина', 'алиса', 'алия', 'алкибия', 'алкимена', 'алкмена', 'алла', 'аллекто', 'алона', 'альба', 'альберта', 'альбертина', 'альбина', 'алькандра', 'альма', 'альмас', 'альфира', 'аля', 'амалия', 'амаль', 'амани', 'амата', 'аматулла', 'амели', 'амина', 'амира', 'амна', 'амфирея', 'амфитрита', 'анан', 'анания', 'анастасия', 'анатолия', 'анбар', 'ангелина', 'анета', 'анжела', 'анжелика', 'аниса', 'анисия', 'анисья', 'анита', 'анна', 'антиклея', 'антиопа', 'антонида', 'антонина', 'антония', 'антуанетта', 'анфиса', 'анфия', 'аполлинария', 'апраксия', 'апфия', 'арабелла', 'арва', 'арета', 'арибах', 'аридж', 'арина', 'аркадия', 'арсения', 'артемида', 'артемия', 'аруб', 'архелая', 'асалах', 'асах', 'асилях', 'асия', 'аскитри', 'аскитрия', 'асклиада', 'асклипиодота', 'аслия', 'аспазия', 'астинома', 'астиоха', 'астра', 'ася', 'атемия', 'афанасия', 'афаф', 'афина', 'афраа', 'афрах', 'афродита', 'ахд', 'ахия', 'ахлям', 'аэлита', 'аэлла', 'баасыма', 'бадрийя', 'бажена', 'балкис', 'банан', 'бану', 'басма', 'бастет', 'басыма', 'батуль', 'батэйя', 'бахийя', 'башиира', 'башира', 'беата', 'беатриса', 'бела', 'белла', 'берислава', 'берта', 'бика', 'бланка', 'бланш', 'богдана', 'божана', 'болеслава', 'борислава', 'боряна', 'бояна', 'бранислава', 'бремуза', 'брисеида', 'бронислава', 'бусайна', 'бушра', 'валентина', 'валериана', 'валерия', 'валиулла', 'валия', 'валя', 'ванда', 'варвара', 'василина', 'василиса', 'василисса', 'васса', 'вата', 'вафа', 'вацлава', 'вевея', 'вега', 'веда', 'вела', 'велислава', 'венера', 'вениамина', 'вера', 'вероника', 'веселина', 'веста', 'вивиан', 'видана', 'викторина', 'виктория', 'вилена', 'виола', 'виргиния', 'виринея', 'вита', 'виталина', 'виталия', 'влада', 'владилена', 'владимира', 'владислава', 'владлена', 'власта', 'всеслава', 'гаафа', 'габриэлла', 'гада', 'гаиания', 'гайда', 'гайша', 'гайя', 'галатея', 'гали', 'галима', 'галина', 'галия', 'галя', 'ганна', 'гасун', 'гаяна', 'геба', 'гезиона', 'гекамеда', 'гекуба', 'гелена', 'гелия', 'гелла', 'генриетта', 'гера', 'герда', 'гермиона', 'гертруда', 'гизела', 'гипподамия', 'гипполита', 'гира', 'глафира', 'гликерия', 'глория', 'голиндуха', 'голуба', 'горислава', 'грация', 'груня', 'груша', 'гуль', 'гульзар', 'гульнара', 'дайна', 'дана', 'дания', 'дарена', 'дарина', 'дария', 'дарья', 'дарьяна', 'дафния', 'дебора', 'дейзи', 'декабрина', 'делия', 'дельфина', 'деримахия', 'дериона', 'дея', 'джаля', 'джамиля', 'джанан', 'джемила', 'джемма', 'джина', 'джоконда', 'джульетта', 'джумана', 'диана', 'дидамия', 'дидона', 'дилара', 'дильбара', 'дина', 'динара', 'диодора', 'дионисия', 'добрава', 'долорес', 'доминика', 'домна', 'домника', 'домнина', 'дора', 'доротея', 'дорофея', 'драгомира', 'дросида', 'дуклида', 'духа', 'ева', 'еванфия', 'еввула', 'евгения', 'евдокия', 'евдоксия', 'евлалия', 'евлампия', 'евникия', 'евпейф', 'евпраксия', 'евсевия', 'евстолия', 'евтихия', 'евтропия', 'евфалия', 'евфимия', 'евфрасия', 'евфросиния', 'евфросинья', 'екатерина', 'елена', 'елизавета', 'еликонида', 'елима', 'елисавета', 'еннава', 'епистимия', 'епихария', 'ермиония', 'еротиида', 'есения', 'есфирь', 'ефимия', 'ефросиния', 'ефросинья', 'жанна', 'ждана', 'женевьева', 'жизель', 'жозефина', 'жюльетта', 'забира', 'заира', 'зайна', 'зайнаб', 'зайнулла', 'зайтуна', 'закийя', 'закия', 'замира', 'замфира', 'зара', 'зарина', 'зафира', 'захира', 'захра', 'захрах', 'звенислава', 'зиля', 'зина', 'зинаида', 'зиннат', 'зиновия', 'зита', 'зифа', 'зия', 'злата', 'зоиса', 'зоя', 'зульфия', 'зухра', 'ибтихадж', 'иванна', 'иветта', 'ида', 'идея', 'идофея', 'изабелла', 'издихар', 'изида', 'изис', 'изольда', 'изот', 'икрима', 'илария', 'илия', 'ильхам', 'имтисаль', 'инайя', 'инам', 'инас', 'инга', 'индира', 'инес', 'инесса', 'инна', 'интисар', 'иоанна', 'иовилла', 'иоланта', 'иона', 'иосия', 'ипатия', 'ипполита', 'ираида', 'ираклия', 'ирида', 'ирина', 'ироида', 'исидора', 'искра', 'иулиания', 'иулита', 'иулия', 'иуния', 'иустина', 'ифигения', 'ифимедея', 'ифиноя', 'ифиона', 'иштар', 'ия', 'каздоя', 'казимира', 'кайета', 'калерия', 'каликэ', 'калина', 'калипсо', 'калиса', 'калисса', 'калисфения', 'каллиста', 'камелия', 'камилла', 'камиля', 'камля', 'капитолина', 'карима', 'карина', 'кармен', 'каролина', 'касима', 'касиния', 'кассандра', 'катада', 'катерина', 'катиба', 'катя', 'керкира', 'кибела', 'кикилия', 'килисса', 'киматоя', 'киприда', 'кира', 'кириакия', 'кириена', 'кирилла', 'кифа', 'клавдия', 'клара', 'клариса', 'клеопа', 'клеопатра', 'климена', 'климентина', 'клитемнестра', 'клония', 'коломбина', 'конкордия', 'констанция', 'кора', 'корделия', 'крискентия', 'кристина', 'ксана', 'ксения', 'кульсум', 'кяукаб', 'кяусар', 'лавиния', 'лада', 'лайла', 'лала', 'лампеция', 'лаодамия', 'лаодика', 'латона', 'латыфа', 'лаура', 'левкофея', 'леда', 'лейла', 'лена', 'лениана', 'ленина', 'леонида', 'леонила', 'леонора', 'леонтия', 'лера', 'леся', 'ливия', 'лидия', 'лилиана', 'лилия', 'лина', 'линда', 'лия', 'лола', 'лолита', 'лора', 'луиза', 'лукерия', 'лукерья', 'лукиана', 'лукина', 'лукия', 'лукреция', 'лутфулла', 'любаба', 'любава', 'любовь', 'любомира', 'людвига', 'людмила', 'люси', 'люция', 'ляма', 'лямис', 'лямья', 'маат', 'мавийя', 'мавлюда', 'мавра', 'магда', 'магдалина', 'маджида', 'маджиида', 'мадина', 'мадиха', 'май', 'маййяда', 'маймуна', 'майса', 'майсун', 'майя', 'макарим', 'макрина', 'маланто', 'маланья', 'малахия', 'малика', 'мальвина', 'маляк', 'мамелхва', 'мамету', 'мамика', 'маналь', 'манар', 'манефа', 'марам', 'маргарита', 'мариана', 'марианна', 'марина', 'марионилла', 'мария', 'марлена', 'марта', 'марфа', 'марьям', 'мастридия', 'матильда', 'матрена', 'матрона', 'маха', 'маша', 'мая', 'мегара', 'медея', 'мелания', 'мелисса', 'мелитина', 'мерседес', 'милада', 'милана', 'милица', 'милослава', 'миляуша', 'мина', 'минерва', 'минодора', 'мира', 'мирабелла', 'миранда', 'миропия', 'мирослава', 'мирра', 'митродора', 'млада', 'моико', 'моника', 'мстислава', 'муавийя', 'муза', 'муна', 'мунира', 'муфида', 'мухджа', 'мушира', 'набиля', 'набиха', 'навсикая', 'нада', 'надва', 'надежда', 'наджат', 'наджах', 'наджва', 'наджиба', 'наджия', 'наджйя', 'наджля', 'надида', 'надира', 'надия', 'надья', 'назаха', 'назийя', 'назира', 'назиха', 'наиля', 'наима', 'наина', 'найля', 'нана', 'насира', 'настасья', 'настя', 'насыха', 'наталия', 'наталья', 'науваль', 'наувар', 'науфаль', 'нафиса', 'нахля', 'нашида', 'нашита', 'некиппа', 'нелли', 'неонила', 'неонилла', 'нерина', 'несаим', 'неэра', 'нибааль', 'нида', 'ника', 'никиппа', 'нимаат', 'нимфодора', 'нина', 'ниназу', 
    'нинелла', 'нинель', 'нинон', 'нинсуи', 'ниоба', 'нирса', 'новелла', 'нонна', 'нора', 'нузар', 'нузха', 'нунехия', 'нур', 'нури', 'нурулла', 'нусайба', 'нут', 'нуф', 'нуха', 'ойгуль', 'оксана', 'октавия', 'октябрина', 'олеся', 'оливия', 'олимпиада', 'олимпия', 'ольга', 'оля', 'осия', 'офелия', 'павла', 'павлина', 'паллада', 'палома', 'пальмира', 'памела', 'пандора', 'параскева', 'патриция', 'паула', 'пелагея', 'пелагия', 'пенелопа', 'пенфесилия', 'перпетуя', 'персефона', 'петрония', 'пиама', 'пинна', 'платонида', 'полактия', 'полетта', 'полидамна', 'поликсена', 'поликсения', 'полина', 'поплия', 'правдина', 'прасковья', 'препедигна', 'преслава', 'прискилла', 'проскудия', 'пульхерия', 'рабаб', 'рабиа', 'рабига', 'равийя', 'равула', 'рагд', 'рада', 'раджа', 'радмила', 'радуа', 'радыйя', 'раида', 'раиса', 'раймонда', 'райя', 'ракия', 'рана', 'ранд', 'ранийя', 'рауда', 'рафа', 'рахиль', 'раша', 'рашида', 'ревекка', 'ревмира', 'регина', 'рената', 'рея', 'рида', 'риза', 'рим', 'рима', 'римма', 'рипсимия', 'рита', 'рогенда', 'рогнеда', 'роза', 'розалия', 'розана', 'розмари', 'роксана', 'роланда', 'ростислава', 'рувайда', 'рукайя', 'рукан', 'русана', 'руслана', 'руфина', 'руфь', 'саалима', 'сабина', 'сабира', 'савда', 'савсан', 'сайфулла', 'сакина', 'салима', 'салиха', 'саломея', 'сальва', 'сальма', 'самаах', 'самар', 'самийя', 'самира', 'самиха', 'самойла', 'сана', 'санаа', 'сания', 'сара', 'сараа', 'сарийя', 'сарра', 'сафа', 'сафия', 'сахар', 'сахля', 'света', 'светлана', 'светлозара', 'светозара', 'светослава', 'свобода', 'святослава', 'севастиана', 'севастьяна', 'северина', 'селена', 'селестина', 'сельвия', 'сельма', 'семирамида', 'серафима', 'сехмет', 'сешат', 'сибиллы', 'сивилла', 'сидури', 'силили', 'сильва', 'симона', 'синклитикия', 'сира', 'сихам', 'славяна', 'смарагда', 'снандулия', 'снежана', 'соломея', 'соломония', 'сосанна', 'сосипатра', 'софия', 'софония', 'софья', 'станимира', 'станислава', 'стелла', 'степанида', 'стефанида', 'стефания', 'сумайя', 'сурайя', 'сусанна', 'суха', 'сухайля', 'сухайма', 'сухайр', 'сюзанна', 'тавифа', 'таира', 'таисия', 'талия', 'тамазур', 'тамара', 'тара', 'таруб', 'татиана', 'татьяна', 'тахира', 'текла', 'текмесса', 'тереза', 'тина', 'тиро', 'тома', 'трифена', 'трояна', 'уаджиха', 'уалентина', 'уарда', 'уафа', 'уафика', 'уафия', 'уидад', 'уиджан', 'уирка', 'уисааль', 'улита', 'ульфат', 'ульяна', 'умара', 'умида', 'ундина', 'урания', 'урсула', 'устинья', 'фаатин', 'фавста', 'фадуа', 'фадыля', 'фаиза', 'фаина', 'фаляк', 'фарида', 'фариха', 'фатима', 'фатина', 'фатыма', 'фаузия', 'фаустина', 'фаэтуза', 'феба', 'феврония', 'февронья', 'федора', 'федосья', 'фекла', 'феклиста', 'фекуса', 'феликсана', 'фелицата', 'фелицитата', 'фелиция', 'феодора', 'феодосия', 'феодотия', 'феодулия', 'феозва', 'феоктиста', 'феона', 'феопистия', 'феофания', 'феофила', 'фервуфа', 'фермодесса', 'фессалоникия', 'фетида', 'фива', 'фивея', 'фида', 'филадельфия', 'филина', 'филиппия', 'филомена', 'филонилла', 'фирдаус', 'фируза', 'фиряль', 'флавия', 'флора', 'флорентина', 'флоренция', 'флориана', 'фомаида', 'фооса', 'фотина', 'фредерика', 'фрида', 'хаадия', 'хавля', 'хадиджа', 'хадийя', 'хадича', 'хадия', 'хайрийя', 'хайфа', 'хайям', 'хайят', 'халида', 'халима', 'халя', 'хамида', 'хана', 'ханан', 'ханийя', 'ханифа', 'хариесса', 'харита', 'харитина', 'хасна', 'хая', 'хельга', 'хесса', 'хинд', 'хиония', 'хлорида', 'хлоя', 'хриса', 'хрисеида', 'хрисия', 'христина', 'христодула', 'худа', 'хулюк', 'хума', 'хурийя', 'хусн', 'цвета', 'целеиа', 'цецелия', 'цецилия', 'цирцея', 'чеслава', 'чулпан', 'шаадия', 'шарифа', 'шарлотта', 'шатха', 'ширин', 'шушаника', 'эвандра', 'эвелина', 'эвридика', 'эвриклея', 'эвринома', 'эдита', 'эдна', 'электра', 'элеонора', 'элиза', 'элина', 'эльвина', 'эльвира', 'эльза', 'эльмира', 'эмилиан', 'эмилия', 'эмма', 'энона', 'эпикаста', 'эрида', 'эрика', 'эриния', 'эсмеральда', 'эстер', 'эя', 'юдифь', 'юлиана', 'юлиания', 'юлита', 'юлитта', 'юлия', 'юманита', 'юмн', 'юния', 'юнона', 'юсраа', 'юстина', 'ядвига', 'яна', 'ярослава', 'ясира', 'ясмин', 'яфьях']

In [19]:
with open('расширенный - вордстат.txt', 'r', encoding = 'UTF-8') as f:
    wrds = f.readlines()
    stops_ws = set(wrds)

with open('расширенный список.txt', 'r', encoding = 'UTF-8') as fl:
    wrds_1 = fl.readlines()
    stops_wr = set(wrds_1)

In [87]:
stops_big = set.union(stops, stops_y, stops_ws, stops_wr, stops_antela, stops_names_w_low, stops_names_w_low, stops_new)

In [21]:
len(stops_big)

4076

In [22]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops_big]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

In [23]:
data['content_norm'] = data['content'].apply(normalize)

In [24]:
#data['title_norm'] = data['title'].apply(normalize)

In [25]:
#data['title_norm'].head(10)

In [26]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(20)

0     [стандарт, источник, фгоса, васильев, результат, предмет, школа, письмо, произведение, содержание]
1                            [красота, глаз, отчаяние, порыв, кошка, предмет, дело, руина, место, мечта]
2                         [пепеляев, юзеф, книга, якутия, восстание, год, война, место, сибирь, пустынь]
3                          [гонка, команда, пилот, сезон, время, машина, формула, место, круг, гран-при]
4                       [есенин, поэт, клюев, год, жизнь, смерть, человек, борода, мариенгоф, революция]
5             [медицина, кафедра, выпускник, уровень, работа, практика, ординатор, рудна, год, обучение]
6                     [книга, человек, мозг, говор, анатомия, глава, звезда, вопрос, слово, азербайджан]
7             [ирак, война, американец, войско, структура, принцип, взрыв, центр, террорист, количество]
8          [нейросеть, клетка, интеллект, время, основа, изображение, задача, работа, система, внимание]
9            [россия, вступление, кристалина, георгиев,

In [27]:
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.13
Recall -  0.26
F1 -  0.17
Jaccard -  0.1


#### Итак, способ №1: увеличить список стоп-слов

### Воспользуемся TfidfVectorizer.

In [28]:
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [111]:
#data['content_norm_str'] = data['content']

In [29]:
tfidf = TfidfVectorizer(min_df=6, ngram_range=(1,2))

In [30]:
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=6,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [31]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

In [32]:
texts_vectors = tfidf.transform(data['content_norm_str'])

In [36]:
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 

In [37]:
keywords[:3]

[['стандарт',
  'васильев',
  'источник',
  'предмет',
  'произведение',
  'письмо',
  'редакция',
  'содержание',
  'школа',
  'литература'],
 ['красота',
  'отчаяние',
  'порыв',
  'кошка',
  'глаз',
  'руина',
  'предмет',
  'занавеска',
  'мечта',
  'безобразие'],
 ['якутия',
  'восстание',
  'книга',
  'сибирь',
  'красных',
  'перо',
  'война',
  'пустынь',
  'мороз',
  'большевик']]

In [38]:
evaluate(data['keywords'], keywords)

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09


## Попробуем графы!

In [78]:
from itertools import combinations
import networkx as nx

In [159]:
def get_kws(text, top=5, window_size=3, random_p=0.1):

    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        #print(window)
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    # нормализуем строки, чтобы получилась вероятность перехода
    for i in range(m.shape[0]):
        m[i] /= np.sum(m[i])
    
    # случайно выберем первое слова, а затем будет выбирать на основе полученых распределений
    # сделаем так 5 раз и добавим каждое слово в счетчик
    # чтобы не забиться в одном круге, иногда будет перескакивать на случайное слово
    
    c = Counter()
    # начнем с абсолютного случайно выбранного элемента
    n = np.random.choice(len(vocab))
    for i in range(500): # если долго считается, можно уменьшить число проходов
        
        # c вероятностью random_p 
        # перескакиваем на другой узел
        go_random = np.random.choice([0, 1], p=[1-random_p, random_p])
        if go_random:
            n = np.random.choice(len(vocab))
        
        n = take_step(n, m)
        # записываем узлы, в которых были
        c.update([n])
    
    # вернем топ-N наиболее часто встретившихся сл
    return [id2word[i] for i, count in c.most_common(top)]

def take_step(n, matrix):
    rang = len(matrix[n])
    # выбираем узел из заданного интервала, на основе распределения из матрицы совстречаемости
    next_n = np.random.choice(range(rang), p=matrix[n])
    return next_n

In [164]:
%%time
keywords_rw = data['content_norm'].apply(lambda x: get_kws(x, 5, 15))

CPU times: user 3min 6s, sys: 484 ms, total: 3min 7s
Wall time: 3min 6s


In [165]:
evaluate(data['keywords'], keywords_rw)

Precision -  0.16
Recall -  0.15
F1 -  0.15
Jaccard -  0.09


In [191]:
def build_matrix(text, window_size=10):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))


    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    #print(m)
    return m, id2word

def some_centrality_measure(text, window_size=10, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    # тут можно поставить любую метрику
    node2measure = dict(nx.pagerank(G))

    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [192]:
%%time
keyword_nx = data['content_norm'].apply(lambda x: some_centrality_measure(x, 10, 10))

CPU times: user 5min 4s, sys: 0 ns, total: 5min 4s
Wall time: 5min 4s


In [193]:
evaluate(data['keywords'], keyword_nx)

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09


In [194]:
keyword_nx.head(10)

0    [стандарт, источник, васильев, результат, фгоса, предмет, произведение, письмо, содержание, школа]
1                           [красота, глаз, отчаяние, кошка, порыв, руина, дело, предмет, место, мечта]
2                          [пепеляев, книга, юзеф, якутия, война, восстание, год, место, сибирь, мороз]
3                         [команда, гонка, пилот, сезон, время, машина, круг, формула, место, гран-при]
4                           [есенин, поэт, клюев, год, жизнь, смерть, человек, мариенгоф, борода, стих]
5     [медицина, кафедра, выпускник, уровень, работа, практика, рудна, ординатор, тиаго, преподаватель]
6                    [книга, человек, мозг, говор, анатомия, глава, звезда, слово, азербайджан, вопрос]
7                    [ирак, война, американец, войско, структура, штаб, бригада, узел, нападение, сила]
8         [нейросеть, клетка, интеллект, время, изображение, основа, задача, работа, внимание, система]
9           [россия, кристалина, георгиев, вступление, вто, банк

In [ ]:
def build_graph(text):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))
    
    G = nx.MultiGraph()
    

In [ ]:
def dens_meth(core):
    

## Вариант № 2
#### использовать для нормализации mystem

In [116]:
from nltk.tokenize import word_tokenize
import string
import re
import collections
import math

In [88]:
def preprocessing(input_text, del_stopwords=True, del_digit=True):
    """
    :input: raw text
        1. lowercase, del punctuation, tokenize
        2. normal form
        3. del stopwords
        4. del digits
    :return: lemmas
    """
    russian_stopwords = set(stopwords.words('russian'))
    if del_digit:
        input_text = re.sub('[0-9]', '', input_text)
    words = [x.lower().strip(string.punctuation+'»«–…') for x in word_tokenize(input_text)]
    lemmas = []
    for word in words:
        word_analized = mystem.analyze(word)
        #print(word_analized)
        if word_analized != []:
            if 'analysis' in word_analized[0]:
                #print(word_analized[0])
                if word_analized[0]['analysis'] != []:
                    if 'S' in word_analized[0]['analysis'][0]['gr']:

                        lemmas.append(word_analized[0]['analysis'][0]['lex'])
    
    
    lemmas_arr = []
    for lemma in lemmas:
        #print(lemma)
        if not lemma in stops_big:
            #print(lemma)
            lemmas_arr.append(lemma)
        #else:
            #print(lemma)
    return lemmas_arr

In [89]:
data['content_norm_myst'] = data['content'].apply(preprocessing)

In [90]:
data['content_norm_myst'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(20)

0     [фгос, стандарт, источник, васильева, результат, предмет, школа, письмо, произведение, содержание]
1                            [красота, глаз, отчаяние, порыв, кошка, предмет, дело, руины, место, мечта]
2                    [пепеляев, юзефович, книга, якутия, восстание, война, место, сибирь, пустыня, жанр]
3                              [команда, гонка, пилот, сезон, время, машина, формула, кими, место, круг]
4                           [есенин, поэт, клюев, жизнь, смерть, человек, борода, стих, революция, друг]
5          [медицина, кафедра, выпускник, уровень, работа, ординатор, рудна, обучение, практика, диплом]
6                          [книга, человек, мозг, говор, анатомия, глава, вопрос, слово, м, азербайджан]
7             [ирак, война, американец, войско, структура, принцип, взрыв, центр, террорист, количество]
8          [нейросеть, клетка, интеллект, время, основа, изображение, задача, работа, система, внимание]
9      [россия, вступление, кристалина, вто, георгиев, 

In [93]:
evaluate(data['keywords'], data['content_norm_myst'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.14
Recall -  0.27
F1 -  0.18
Jaccard -  0.1


#### УРА! Стало еще лучше!

### А что если и tf-idf чуть изменить?

In [97]:
data['content_norm_myst_str'] = data['content_norm_myst'].apply(' '.join)

In [98]:
#data['content_norm_myst_str'][:2]

0    среда заседание стандарт фгос министерство образование наука рф собрание понедельник просьба член сообщение общественник согласие позиция министерство фгос заседание фгос школа март косультация фгос школа гильдия словесник письмо имя министр образование наука рф васильева мнение автор письмо фгос право ребенок стандарт го класс стандарт министерство образование право орган ребенок незнание список произведение обучение возможность освоение риск разговор урок автор письмо произведение класс ученик класс возраст ребенок номер класс заседание фгос беседа обозреватель нга васильева претензия гильдия словесник число книга господин волк текст письмо страничка учитель язык литература волк нга письмо количество произведение стихотворение васильева цифра сонм романов количество произведение стих вещь память душа интеллект вкус список момент министр беседа нга процесс доработка стандарт литература концепция стандарт структура совокупность требование результат реализация программа изменение с

In [204]:
tfidf = TfidfVectorizer()

In [205]:
texts_vectors_myst = tfidf.fit_transform(data['content_norm_myst_str'])

In [206]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

In [102]:
#texts_vectors_myst = tfidf.transform(data['content_norm_myst_str'])

In [207]:
keywords_myst = [[id2word[w] for w in top] for top in texts_vectors_myst.toarray().argsort()[:,:-11:-1]] 

In [208]:
keywords[:3]

[['стандарт',
  'васильев',
  'источник',
  'предмет',
  'произведение',
  'письмо',
  'редакция',
  'содержание',
  'школа',
  'литература'],
 ['красота',
  'отчаяние',
  'порыв',
  'кошка',
  'глаз',
  'руина',
  'предмет',
  'занавеска',
  'мечта',
  'безобразие'],
 ['якутия',
  'восстание',
  'книга',
  'сибирь',
  'красных',
  'перо',
  'война',
  'пустынь',
  'мороз',
  'большевик']]

In [209]:
evaluate(data['keywords'], keywords)

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09


In [210]:
def smt(text, i):
    return list(texts_vectors_myst.toarray()[i])

In [211]:
data['tf_idf'] = data.apply(lambda _: '', axis=1)

In [ ]:
i = 0
result = []
for text in data['content_norm']:
    data['tf_idf'][i] = smt(text, i)
    #print(data['tf_idf'][i])
    #result.append(keys)
    i += 1

In [ ]:
def build_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    return m, id2word

def some_centrality_measure(text, i, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    # тут можно поставить любую метрику
    node2measure = dict(nx.degree(G))

    for key in node2measure.keys():
        try:
            tf_idf = data['tfidf'][i][id2word[key]]
        except:
            tf_idf = 0
        print(id2word[key], str(tf_idf))
        node2measure[key] = node2measure[key]/len(G)*tf_idf
    #print(node2measure)
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]